# Using the EconModelClass

`EconModel` is a small package for easily working with economic models in Python. It has four objectives:

1. Make it easy to write well-structed code.
1. Provide standard functionality for copying, saving and loading.
1. Provide an easy interface to call [numba](http://numba.pydata.org/) JIT compilled functions.
1. Provide an easy interface to call C++ functions.

The two first objectives should be of interest to everybody, while the two last objectives are for more advanced users.

This notebook provides a simple example for using the `EconModelClass`. See documentation in [EconModel/README.md](https://github.com/NumEconCopenhagen/EconModel#econmodel).

In [1]:
%load_ext autoreload
%autoreload 2

import shutil
import numpy as np
import numba as nb

from EconModel import EconModelClass, jit

# Baseline Usage

The model is **required** to have the following three methods:

1. `.settings()`: Choose fundamental settings.
1. `.setup()`: Set free parameters.
1. `.allocate()`: Set compound parameters and allocate arrays.

When the model is initialized `.settings()`, `.setup()` and `.allocate()` are all called (in that order).

Per default the namespaces `.par` (for parameters), `.sol` (for solution objects) and `.sim` (for simulation objects) are available. The philosophy of the package is that all the content of the model is contained in these namespaces. After the model has been initialized, no additional entries should be added to the namespaces, and each entry should only change *value(s)* (or shape for arrays), but never *type* (or number of dimensions for arrays).

In [2]:
class MyModelClassBasic(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
            
        pass
    
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        
        par.X = np.linspace(0,10,par.N)
        par.Y = np.zeros(par.N)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        par.Y[:] = par.X*(par.a+par.b)


In `.settings()` the user can change the default behavior by specifying: 

1. `self.savefolder = str`: Filepath to save in and load from (default: saved).
1. `self.namespaces = [str]`: List of namespaces (in addition to `.par`, `.sol`, `.sol`).
1. `self.other_attrs = [str]`: List of additional attributes to be copied and saved.


## Setup

In [3]:
model = MyModelClassBasic(name='example')
model.solve()
print(model)

Modelclass: MyModelClassBasic
Name: example

namespaces: ['sol', 'par', 'sim']
other_attrs: []
savefolder: saved
not_floats: None

sol:
 memory, gb: 0.0

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



**Updating at initialization:**

In [4]:
model_alt = MyModelClassBasic(name='alt',par={'a':3.0,'b':1.2})
print(model_alt)

Modelclass: MyModelClassBasic
Name: alt

namespaces: ['sol', 'par', 'sim']
other_attrs: []
savefolder: saved
not_floats: None

sol:
 memory, gb: 0.0

par:
 N = 10 [int]
 a = 3.0 [float]
 b = 1.2 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



**Unpack for later use:**

In [5]:
par = model.par

## Copy, save and load

In [6]:
model_copy = model.copy(name='example_copy')
model_copy.save()
model_loaded = MyModelClassBasic(name='example_copy',load=True)
print(model_loaded)

Modelclass: MyModelClassBasic
Name: example_copy

namespaces: ['sol', 'par', 'sim']
other_attrs: []
savefolder: saved
not_floats: None

sol:
 memory, gb: 0.0

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



## To and from dictionary 

In [7]:
modeldict = model.as_dict()
model_dict = MyModelClassBasic(name='example_dict',from_dict=modeldict)
print(model_dict)

Modelclass: MyModelClassBasic
Name: example_dict

namespaces: ['sol', 'par', 'sim']
other_attrs: []
savefolder: saved
not_floats: None

sol:
 memory, gb: 0.0

par:
 N = 10 [int]
 a = 2.0 [float]
 b = 1.0 [float]
 threads = 4 [int]
 txt = a [str]
 txtlist = N|threads [str]
 X = ndarray with shape = (10,) [dtype: float64]
 Y = ndarray with shape = (10,) [dtype: float64]
 memory, gb: 0.0

sim:
 memory, gb: 0.0



# Numba and C++

To call **numba-compiled** functions or **C++ functions** it is necessary to infer the type of all elements in all namespaces. To ensure this is done correctly, the user must specify `.not_floats = [str]` in `.settings()` as a list of all scalar elements in the namespaces, which are not floats, but e.g. have type `bool`, `int` or `str`.

The extended model specification thus looks:

In [8]:
class MyModelClass(EconModelClass):
    
    # __init__ is inherited from EconModelClass
    
    def settings(self): # required
        """ choose settings """
        
        # optional
        self.namespaces = [] # in addition to .par, . sol, .sim
        self.savefolder = 'saved' # folder to save model in (default: 'saved')
        
        # only if using numba or C++
        self.not_floats = ['N','threads'] # non-float scalar elements in namespaces (for safe type inference)
        
        # only if using C++
        self.cpp_filename = 'cppfuncs/example.cpp' # required for using C++
        self.cpp_options = {'compiler':'vs'} # optional
        self.cpp_structsmap = {'par':'par_struct'} # optional
        
    def setup(self): # required
        """ set free parameters """
        
        par = self.par
        
        par.N = 10
        par.a = 2.0
        par.b = 1.0
        par.threads = 4
        par.txt = 'a'
        par.txtlist = 'N|threads'
        
    def allocate(self): # required
        """ set compound parameters and allocate arrays """
        
        par = self.par
        
        par.X = np.linspace(0,10,par.N)
        par.Y = np.zeros(par.N)
    
    def solve(self): # user-defined
        """ solve the model"""
        
        par = self.par
        par.Y[:] = par.X*(par.a+par.b)


In [9]:
model = MyModelClass(name='numba')

## Numba

**Goal:** Call [numba](http://numba.pydata.org/) JIT compilled function easily.

**Under-the-hood:** `with jit(model) as model_jit:` temporarily turns all `namespaces` into `namedtuples`, which can be used as input in JIT compilled functions. Fields of namedtuples are immutable.

In [10]:
@nb.njit
def fun(par):
    par.Y[:] = par.X*(par.a+par.b)

In [11]:
def test_numba(model):
    with jit(model) as model_jit:
        fun(model_jit.par)

In [12]:
%time test_numba(model)
%time test_numba(model)

Wall time: 992 ms
Wall time: 0 ns


**Note:** The first run is slower due to compiling.

In [13]:
assert np.allclose(par.X*(par.a+par.b),par.Y)

## Advanced: Recompilling

When a new model is created, re-compilation of jit-functions are usually necessary:

In [14]:
model_new = MyModelClass(name='numba_new')
%time test_numba(model_new)
%time test_numba(model_new)

Wall time: 147 ms
Wall time: 0 ns


This is *not* the case, when a copy is made instead:

In [15]:
model_new_ = model_new.copy(name='numba_new_copy')
%time test_numba(model_new)

Wall time: 949 µs


## Calling C++

**Platform:** Only works on *Windows*.

**Compilers:** One of these compilers must be installed

* **vs**: Free *Microsoft Visual Studio 2022 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Free *Intel one API* ([link](https://www.intel.com/content/www/us/en/developer/tools/oneapi/base-toolkit-download.html))

For parallization we will use **OpenMP**.

**Interface:**

1. all elements in **namespaces** are availible as **C++ structs** with `_struct` appended (default, else specify `.cpp_structsmap` in `.settings()`).
2. `cpptools.link_to_cpp()` is called by `.link_to_cpp` using `.cpp_filename` and the result is returned in `.cpp`.
3. `cpp_options` is the compiler options.
4. by default `force_compile=True` when calling `.link_to_cpp`.

**Details:** Provided in `02. More on calling C++.ipynb`.

**Link:**

In [16]:
model.link_to_cpp(do_print=True)

Linking to: cppfuncs/example.cpp

### finding all included files ###

par_struct.cpp
example_include.cpp

### writing structs ###

cppfuncs/sol_struct.cpp

cppfuncs/par_struct.cpp
 int N;
 double a;
 double b;
 int threads;
 char* txt;
 char* txtlist;
 double* X;
 double* Y;

cppfuncs/sim_struct.cpp

### analyzing cppfuncs/example.cpp ###

function: fun
return type: void
argument types: ['par_struct*']

function: fun_nostruct
return type: void
argument types: ['double*', 'double*', 'int', 'double', 'double', 'int', 'char*']

### analyzing cppfuncs/par_struct.cpp ###

### analyzing cppfuncs/example_include.cpp ###

function: fun_nostruct_alt
return type: void
argument types: ['double*', 'double*', 'int', 'double', 'double', 'int', 'char*']

function: I_DO_NOTHING
return type: double
argument types: []

### compiling and linking ###

compile.bat:
cd /d "C:/Program Files/Microsoft Visual Studio/2022/Community/VC/Auxiliary/Build/"
call vcvarsall.bat x64
cd /d "C:\Users\gmf123\Documents\rep

**Compile warnings and errors** are shown in the terminal.

**Call C++:** It is now easy to call the C++ function

In [17]:
model.cpp.fun(par)
assert np.allclose(par.X*(par.a+par.b),par.Y)

**Re-compile:**

In [18]:
model.cpp.recompile()

**Multiple models:** 

In [19]:
# a. don't compile when the .dll file is already loaded
model_alt = MyModelClass('testcpp_alt')
model_alt.link_to_cpp(force_compile=False) # default is force_compile = True

# b. function call
par = model_alt.par
model_alt.cpp.fun_nostruct(par.X,par.Y,par.N,par.a,par.b,par.threads,par.txt)
assert np.allclose(par.X*(par.a+par.b),par.Y)

# c. delink without removing the .dll file
model.cpp.delink()

# d. delete model -> automatically calls .cpp.delink()
del model_alt

# e. now we can compile again
model_alt = MyModelClass('testcpp_alt')
model_alt.link_to_cpp()
model_alt.cpp.clean_up()

# Clean-up

In [20]:
shutil.rmtree('saved')